In [70]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.proxy import Proxy, ProxyType
import requests
from dotenv import load_dotenv
from os import environ
import random
import pandas as pd
import time
from fake_useragent import UserAgent

In [71]:
load_dotenv()
random.seed()

In [72]:
path = environ["WEBDRIVER_PATH"]
url = environ["BASE_URL"]
feed_endpoint = environ["FEED_ENDPOINT"]
login_uid_xpath = '//*[@id="session_key"]' 
login_password_xpath = '//*[@id="session_password"]'
login_submit_xpath = '//*[@id="main-content"]/section[1]/div/div/form/button'

In [73]:
phone_skip_btn = "/html/body/div/div[1]/section/div[2]/div/article/footer/div/div/button"

In [74]:
options = Options()
ua = UserAgent()
user_agent = ua.random

options.add_argument(f'user-agent={user_agent}')
driver = webdriver.Chrome(
        # options=options, 
        executable_path=path
    )
driver.get(url)
uid_field = driver.find_element(By.XPATH, login_uid_xpath)
uid_field.send_keys(environ["LINKEDIN_USERNAME"])
password_field = driver.find_element(By.XPATH, login_password_xpath)
password_field.send_keys(environ["LINKEDIN_PASSWORD"])
okay_btn = driver.find_element(By.XPATH, login_submit_xpath).click()
if environ["ADD_PHONE_ENDPOINT"] in driver.current_url:
    btn = driver.find_element(By.XPATH, phone_skip_btn)
    btn.click()
    btn = None
time.sleep(5)
driver.quit()

/tmp/ipykernel_14789/4189617847.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


In [10]:
with open("../data_store/uids.csv", "rt", encoding="utf-8") as data:
    uid_list = data.read().split("\n")
uid_list.sort()
print(uid_list)

['antarasaikia', 'nabajyoti-borah-944000160', 'parthava-borah-98253713a', 'prithoo11335', 'shikha-sharma-b9362b169', 'tridip-das-b5837911b']
